In [0]:
import requests
import pandas as pd

In [0]:
C=dbutils.widgets.get('COUNT')
API_URL=f'https://dogapi.dog/api/v2/breeds?page%5Bnumber%5D=1&page%5Bsize%5D={C}'

In [0]:
response=requests.get(API_URL)
api_data = response.json()
print(api_data)


{'data': [{'id': '036feed0-da8a-42c9-ab9a-57449b530b13', 'type': 'breed', 'attributes': {'name': 'Affenpinscher', 'description': 'The Affenpinscher is a small and playful breed of dog that was originally bred in Germany for hunting small game. They are intelligent, energetic, and affectionate, and make excellent companion dogs.', 'life': {'max': 16, 'min': 14}, 'male_weight': {'max': 5, 'min': 3}, 'female_weight': {'max': 5, 'min': 3}, 'hypoallergenic': True}, 'relationships': {'group': {'data': {'id': 'f56dc4b1-ba1a-4454-8ce2-bd5d41404a0c', 'type': 'group'}}}}, {'id': 'dd9362cc-52e0-462d-b856-fccdcf24b140', 'type': 'breed', 'attributes': {'name': 'Afghan Hound', 'description': 'The Afghan Hound is a large and elegant breed of dog that was originally bred in Afghanistan for hunting small game. They are intelligent, independent, and athletic, and make excellent companion dogs.', 'life': {'max': 14, 'min': 12}, 'male_weight': {'max': 27, 'min': 23}, 'female_weight': {'max': 25, 'min': 20

In [0]:
pd_df=pd.json_normalize(api_data['data'],sep='_')
pd_df.head()

,id,type,attributes_name,attributes_description,attributes_life_max,attributes_life_min,attributes_male_weight_max,attributes_male_weight_min,attributes_female_weight_max,attributes_female_weight_min,attributes_hypoallergenic,relationships_group_data_id,relationships_group_data_type
0,036feed0-da8a-42c9-ab9a-57449b530b13,breed,Affenpinscher,The Affenpinscher is a small and playful breed...,16,14,5,3,5,3,True,f56dc4b1-ba1a-4454-8ce2-bd5d41404a0c,group
1,dd9362cc-52e0-462d-b856-fccdcf24b140,breed,Afghan Hound,The Afghan Hound is a large and elegant breed ...,14,12,27,23,25,20,False,be0147df-7755-4228-b132-2518c0c6d10d,group
2,1460844f-841c-4de8-b788-271aa4d63224,breed,Airedale Terrier,The Airedale Terrier is a large and powerful b...,14,12,23,20,20,18,False,1bbf373b-1937-4e73-9863-45385daa4979,group
3,e7e99424-d514-4b56-9f0c-05736f6dd22d,breed,Akita,"The Akita is a large, muscular dog breed that ...",12,10,60,35,50,35,False,56081cf0-fdf2-4114-9bf7-23a3f5b6af91,group
4,667c7359-a739-4f2b-abb4-98867671e375,breed,Alaskan Klee Kai,The Alaskan Klee Kai is a small to medium-size...,15,12,7,6,7,6,False,8000793f-a1ae-4ec4-8d55-ef83f1f644e5,group


In [0]:
del pd_df['type'] 
del pd_df['relationships_group_data_id'] 
del pd_df['relationships_group_data_type']
pd_df.head()

,id,attributes_name,attributes_description,attributes_life_max,attributes_life_min,attributes_male_weight_max,attributes_male_weight_min,attributes_female_weight_max,attributes_female_weight_min,attributes_hypoallergenic
0,036feed0-da8a-42c9-ab9a-57449b530b13,Affenpinscher,The Affenpinscher is a small and playful breed...,16,14,5,3,5,3,True
1,dd9362cc-52e0-462d-b856-fccdcf24b140,Afghan Hound,The Afghan Hound is a large and elegant breed ...,14,12,27,23,25,20,False
2,1460844f-841c-4de8-b788-271aa4d63224,Airedale Terrier,The Airedale Terrier is a large and powerful b...,14,12,23,20,20,18,False
3,e7e99424-d514-4b56-9f0c-05736f6dd22d,Akita,"The Akita is a large, muscular dog breed that ...",12,10,60,35,50,35,False
4,667c7359-a739-4f2b-abb4-98867671e375,Alaskan Klee Kai,The Alaskan Klee Kai is a small to medium-size...,15,12,7,6,7,6,False


In [0]:
df=spark.createDataFrame(pd_df)
df.display()

id,attributes_name,attributes_description,attributes_life_max,attributes_life_min,attributes_male_weight_max,attributes_male_weight_min,attributes_female_weight_max,attributes_female_weight_min,attributes_hypoallergenic
036feed0-da8a-42c9-ab9a-57449b530b13,Affenpinscher,"The Affenpinscher is a small and playful breed of dog that was originally bred in Germany for hunting small game. They are intelligent, energetic, and affectionate, and make excellent companion dogs.",16,14,5,3,5,3,true
dd9362cc-52e0-462d-b856-fccdcf24b140,Afghan Hound,"The Afghan Hound is a large and elegant breed of dog that was originally bred in Afghanistan for hunting small game. They are intelligent, independent, and athletic, and make excellent companion dogs.",14,12,27,23,25,20,false
1460844f-841c-4de8-b788-271aa4d63224,Airedale Terrier,"The Airedale Terrier is a large and powerful breed of dog that was originally bred in England for hunting small game. They are intelligent, energetic, and determined, and make excellent hunting dogs.",14,12,23,20,20,18,false
e7e99424-d514-4b56-9f0c-05736f6dd22d,Akita,"The Akita is a large, muscular dog breed that originated in Japan. They are known for their loyalty and courage.",12,10,60,35,50,35,false
667c7359-a739-4f2b-abb4-98867671e375,Alaskan Klee Kai,The Alaskan Klee Kai is a small to medium-sized breed of dog that was developed in Alaska in the 1970s. It is an active and intelligent breed that is loyal and friendly. The Alaskan Klee Kai stands between 13-17 inches at the shoulder and has a double-coat that can come in various colors and patterns.,15,12,7,6,7,6,false
5328d59b-b4e4-48e9-98ec-0545c66c4385,Alaskan Malamute,"The Alaskan Malamute is a large and powerful sled dog from Alaska. They are strong and hardworking, yet friendly and loyal. Alaskan Malamutes have a thick, double coat that can be any color. They are active and require plenty of exercise and mental stimulation to stay healthy and happy.",15,12,39,34,34,25,false
f72528b5-a5d7-4a17-b709-aba2db722307,American Bulldog,"The American Bulldog is a large and powerful breed of dog that was originally bred in the United States for working on farms. They are intelligent, loyal, and protective, and make excellent guard dogs.",14,12,50,25,45,25,false
4524645f-dda7-4031-9272-dee29f5f91ea,American English Coonhound,"The American English Coonhound is a large and athletic breed of dog that was originally bred in the United States for hunting raccoons. They are intelligent, energetic, and determined, and make excellent hunting dogs.",14,12,30,20,30,20,false
e1c0664d-aa61-4c85-970d-6c86ba197bee,American Eskimo Dog,"The American Eskimo Dog is a small to medium-sized breed with a thick, fluffy coat that comes in white, cream, or biscuit colors. It is known for its intelligence and its ability to learn a wide variety of tricks.",15,12,20,9,20,9,false
8355b9c9-3724-477d-858a-c1c1c0f1743f,American Foxhound,"The American Foxhound is a large and athletic breed of dog that was originally bred in the United States for hunting foxes. They are intelligent, energetic, and determined, and make excellent hunting dogs.",15,12,34,20,34,20,false


In [0]:
df.write.format('delta').option('optimizeWrite', True).mode('append').save('/Volumes/dog_warehouse/default/dog_data/Bronze/Raw_Data')